In [14]:
import geemap
import ee
map = geemap.Map()

In [15]:
nzl = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME','New Zealand'))
# Define the regional bounds for animation frames.
region = nzl.geometry().bounds();

In [16]:
# set start date for analysis
startDate = '2022-01-01';
# set end date for analysis
endDate = '2022-12-31';
#filter the image collection using dates
pptCol = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")\
        .filterDate(startDate, endDate)\
        .select('precipitation');
# Setting year and month as a property of every image which will later be used to form monthly composites
def set_yymm(img):
    return img.set('yymm', img.id().slice(0,6))
pptCol = pptCol.map(set_yymm)
# Define a list of unique observation months from the image collection.
yymmList = ee.List(pptCol.aggregate_array('yymm')).distinct().sort();
# Function to sum up the daily precipitation data of each month 
def groupByMonth(yymm):
    pptMonth = pptCol.filterMetadata('yymm', 'equals', yymm)\
                .sum()\
                .clip(nzl)\
                .set('yymm', yymm);
    return pptMonth
# Map the function over the list of months to build a collection of monthly image composites.
pptMonthList = yymmList.map(groupByMonth);
pptByYYMM = ee.ImageCollection.fromImages(pptMonthList);

In [36]:
pptCol.size().getInfo(), pptByYYMM.size().getInfo()


(364, 12)

In [26]:
# We use 20 year data to construct a baseline figure 
baseCol = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")\
        .filterDate('2000-01-01','2020-01-01')\
        .select('precipitation');
# Create a list of months
months = ee.List.sequence(1, 12);
# Function to find the average monthly precipitation
def avgByMonth(m):
    img = baseCol.filter(ee.Filter.calendarRange(m,m, 'month')).sum().divide(10).clip(nzl);
    return img.set('mm',ee.Number(m).format('%02d'));
# Map over the list of months to build a collection of long term average monthly precipitation composites
baseMonthCol = ee.ImageCollection.fromImages(months.map(avgByMonth))

In [27]:
# Compare the monthly composites against the corresponding baseline to arrive at the deficit figures
def calc_deficit(img):
    
    mm = ee.String(img.get('yymm')).slice(4,6); 
    base = baseMonthCol.filterMetadata('mm','equals',mm).first();
    deficit = base.subtract(img).divide(base).set('yymm', img.get('yymm'));
    return deficit;
deficitByMonth = pptByYYMM.map(calc_deficit);

In [28]:
# Remove deficit of arid areas from visualisation. We define an area as arid if its annual rainfall is less than 500mm.
baseAnnualCol = baseMonthCol.sum();
notArid = baseAnnualCol.gt(500);
def arid_rem(img):
    return img.multiply(notArid);
deficitByMonth = deficitByMonth.map(arid_rem);

In [37]:
# Define RGB visualization parameters.
visParams = {
    'min' : 0,
    'max' : 1,
    'palette': ['e6ffe6','ffffff', 'fff633', 'fc0703']
};

In [30]:
# Create an empty image into which to paint the features.
transparent = ee.Image();
empty = ee.Image.cat([transparent, transparent,transparent]);
# Select and (optionally) rename bands.
empty = empty.select(
    ['constant', 'constant_1', 'constant_2'], #old names
    ['vis-red', 'vis-green', 'vis-blue']  #new names
);
# Paint all the polygon edges with the same number and width, display.
outline = empty.paint(
    featureCollection = nzl,
    color = 1,
    width = 1);

In [31]:
def rgb_fc_mosaic(img):
    #Create RGB visualization images for use as animation frames.
    imgRGB = img.visualize(**visParams).clip(nzl)
#   Mosaic the visualization layers and display (or export).
    mosaic = ee.ImageCollection([imgRGB, outline]).mosaic()
    return mosaic
rgbVis = deficitByMonth.map(rgb_fc_mosaic);

In [32]:
# Define GIF visualization parameters.
gifParams = {
  'region': region,
  'dimensions': 600,
  'crs': 'EPSG:3857',
  'framesPerSecond': 1
};
# Print the GIF URL to the console.
print(rgbVis.getVideoThumbURL(gifParams));

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/f958eac467c62bcb2c93e06ff4c3a5b8-f84ccda3854121e7dac28d6207df4b24:getPixels


In [33]:
import pandas as pd
text = pd.date_range(startDate, endDate, 
              freq='MS').strftime("%b-%Y").tolist()
# Add the months information on each frame of GIF
in_gif = 'nzl.gif'
out_gif = 'nzl_drought_out.gif'
geemap.add_text_to_gif(in_gif, out_gif, xy = ('40%', '0%'), text_sequence = text, font_size=30, font_color = '#ffffff',duration = 1000)

In [34]:
geemap.show_image(out_gif)

Output()